In [3]:
import ECGDataset 
import Models 
import Net
from train_test_validat import *
from self_attention import *
import matplotlib.pyplot as plt
import ecg_plot

import torch
import torch.utils.data as Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import random
import pandas as pd

import time
import os
import gc

random_seed = 2
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)


In [4]:
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [5]:
ALLDataset = ECGDataset.ECG_Dataset_Init('/workspace/data/Preprocess_HTN/data_like_pxl/',filter_age=45)
ALLDataset.report()  # type: ignore
torch.cuda.empty_cache()# 清空显卡cuda
NET = [Net.MLBFNet(True,res = True,se = True,Dropout_rate = 0.25) ] # type: ignore
test_dataset = ECGDataset.ECG_Dataset(ALLDataset.testECGs,ALLDataset.testLabels,ALLDataset.testDf,preprocess= True,num_classes = 0)  # type: ignore


              HTN        NHTN   
   nums       1871      103183  


            orginal            QC         
   nums      105054          65944        
              HTN        NHTN   
   nums       1344      64600   


            orginal   filtered department 
   nums      65944           13953        
              HTN        NHTN   
   nums       1344      12609   


            orginal     removed ages NaN  
   nums      13953           13953        
              HTN        NHTN   
   nums       1344      12609   


            orginal      filtered ages    
   nums      13953            5269        
              HTN        NHTN   
   nums       1344       3925   


            orginal   removed diagnose NaN
   nums       5269            5266        
              HTN        NHTN   
   nums       1344       3922   


              HTN        NHTN   
   ALL        1344       3922   
 testset      346        1606   
 T&V set      998        2316   
              HTN        NHTN 

In [9]:
test_dataset.infos.to_csv('./testdataset.csv')

In [4]:
testmodel = NET[0].to(DEVICE)

In [16]:
for i in range(5):
    testmodel.load_state_dict(torch.load("./model/20221205_081038/parameter_EarlyStoping_"+str(i)+".pt"))
    test_dataloader = Data.DataLoader(dataset=test_dataset, shuffle= False,batch_size=128)
    test_acc = []   
    criterion = torch.nn.CrossEntropyLoss() 
    y_true,y_pred,test_loss,test_acc = eval_model(test_dataloader,criterion,testmodel,DEVICE) # 测试模型
    print('loss =',test_loss,'acc =',test_acc)
    print('f1_macro =',f1_score(y_true, y_pred, average='macro')) 
    print('f1_micro =',f1_score(y_true, y_pred, average='micro')) 
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
    print("Confusion Matrix: ")
    print(cm)
    df_grade = pd.DataFrame(test_dataset.infos['ecgFN'].tolist(),columns=['file'])
    df_grade = pd.concat([df_grade, pd.DataFrame(y_true,columns=['true'])],axis=1)
    df_grade = pd.concat([df_grade, pd.DataFrame(y_pred,columns=['pred'])],axis=1)
    df_grade.to_csv("./model/20221205_081038/answer"+str(i)+".csv")

loss = 0.43475835770368576 acc = 0.87353515625
f1_macro = 0.7934328700956566
f1_micro = 0.8734631147540983
Confusion Matrix: 
[[1460  146]
 [ 101  245]]
loss = 0.45423644967377186 acc = 0.84814453125
f1_macro = 0.7694132650743162
f1_micro = 0.8468237704918032
Confusion Matrix: 
[[1392  214]
 [  85  261]]
loss = 0.4391502160578966 acc = 0.86669921875
f1_macro = 0.7756911714265319
f1_micro = 0.8647540983606559
Confusion Matrix: 
[[1459  147]
 [ 117  229]]
loss = 0.43294546008110046 acc = 0.87548828125
f1_macro = 0.7915174295869827
f1_micro = 0.8755122950819673
Confusion Matrix: 
[[1474  132]
 [ 111  235]]
loss = 0.42531685531139374 acc = 0.88232421875
f1_macro = 0.786521021433993
f1_micro = 0.8796106557377049
Confusion Matrix: 
[[1503  103]
 [ 132  214]]


In [19]:
answer1 = pd.read_csv("./model/20221205_081038/answer0.csv")
answer1.rename(columns={'pred':'pred1'},inplace=True) 
answer2 = pd.read_csv("./model/20221205_081038/answer1.csv")
answer2.rename(columns={'pred':'pred2'},inplace=True) 
answer3 = pd.read_csv("./model/20221205_081038/answer2.csv")
answer3.rename(columns={'pred':'pred3'},inplace=True) 
answer4 = pd.read_csv("./model/20221205_081038/answer3.csv")
answer4.rename(columns={'pred':'pred4'},inplace=True) 
answer5 = pd.read_csv("./model/20221205_081038/answer4.csv")
answer5.rename(columns={'pred':'pred5'},inplace=True) 

answer1 = pd.merge(answer1,answer2.loc[:,['file','pred2']],how='outer',on='file')
answer1 = pd.merge(answer1,answer3.loc[:,['file','pred3']],how='outer',on='file')
answer1 = pd.merge(answer1,answer4.loc[:,['file','pred4']],how='outer',on='file')
answer1 = pd.merge(answer1,answer5.loc[:,['file','pred5']],how='outer',on='file')

In [20]:
answer1

,Unnamed: 0,file,true,pred1,pred2,pred3,pred4,pred5
0,0,21-1-103_34-1_NHTN.npy,0,0,0,0,0,0
1,1,21-1-105_79-0_HTN.npy,1,1,1,1,1,1
2,2,21-1-106_26-0_NHTN.npy,0,0,0,0,0,0
3,3,21-1-108_34-1_NHTN.npy,0,0,0,0,0,0
4,4,21-1-1100_31-0_NHTN.npy,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1947,1947,21-9-459_40-1_NHTN.npy,0,1,0,1,1,1
1948,1948,21-9-461_31-0_NHTN.npy,0,0,0,0,0,0
1949,1949,21-9-463_30-0_NHTN.npy,0,0,0,0,0,0
1950,1950,21-9-65_44-1_NHTN.npy,0,0,0,0,0,0


In [21]:
answer1.insert(loc=len(answer1.columns), column='ERR_Count', value=(abs(5*answer1['true'] -(answer1['pred1']+answer1['pred2']+answer1['pred3']+answer1['pred4']+answer1['pred5']))))

In [22]:
(answer1['ERR_Count']).value_counts()

0    1511
1     128
5     117
3      73
2      68
4      55
Name: ERR_Count, dtype: int64

In [24]:
HTN_errs = answer1[(answer1['true'] == 1)]
HTN_errs['ERR_Count'].value_counts()

0    189
5     64
3     32
1     28
4     18
2     15
Name: ERR_Count, dtype: int64

In [26]:
NHTN_errs = answer1[(answer1['true'] == 0)]
NHTN_errs['ERR_Count'].value_counts()

0    1322
1     100
2      53
5      53
3      41
4      37
Name: ERR_Count, dtype: int64

In [42]:
HTN_errs = answer1[(answer1['ERR_Count']>2)&(answer1['true'] == 1)]

In [43]:
from tqdm import tqdm
csv_data = pd.DataFrame()
count = 0 
for index in tqdm((HTN_errs['Unnamed: 0']).tolist()):
    inf = test_dataset.infos.iloc[index]
    ECG,label = test_dataset.__getitem__(index)
    file_name = inf['name']+'_'+inf['gender']+'_'+str(inf['ages'])+'_'+inf['department'] +'_'+str(inf['diagnose'])+'_'+inf['ID']+'_'+inf['date']
    ecg_plot.plot(ECG*3500/1000, sample_rate = 500, title = file_name,row_height= 10,show_grid=True,show_separate_line=True)
    file_name = str(count)+'_'+str(HTN_errs.loc[index,'file'])+'_'+file_name+'_'+str(HTN_errs.loc[index,'ERR_Count'])
    ecg_plot.save_as_png(file_name,'./ERR_fold/HTN/',dpi = 100)
    csv_data = csv_data.append(inf)
    count = count + 1

100%|██████████| 114/114 [07:13<00:00,  3.80s/it]


In [44]:
csv_data = csv_data.reset_index(drop=True)

In [49]:
csv_data

,num,name,ages,gender,department,diagnose,ID,date,ecgFN,q_sum
0,1191.0,李丽琴,45.0,女,内分泌糖尿病科病房,1.0,848037,2021-01-06 16:57:06,21-1-1191_45-0_HTN.npy,0.0
1,1311.0,李杰,28.0,男,心血管内科病房,1.0,849760,2021-01-19 09:40:16,21-1-1311_28-1_HTN.npy,0.0
2,1472.0,杨国富,54.0,男,心血管内科病房,1.0,851140,2021-01-26 19:29:04,21-1-1472_54-1_HTN.npy,0.0
3,1489.0,杨希,35.0,女,心血管内科病房,1.0,846397,2021-01-06 15:57:22,21-1-1489_35-0_HTN.npy,0.0
4,1948.0,林雪云,35.0,女,内分泌糖尿病科病房,1.0,415049,2021-01-05 15:46:46,21-1-1948_35-0_HTN.npy,0.0
...,...,...,...,...,...,...,...,...,...,...
109,272.0,卢惟铭,51.0,男,内分泌糖尿病科病房,1.0,882533,2021-08-09 15:40:08,21-8-272_51-1_HTN.npy,0.0
110,315.0,叶奕斌,51.0,男,心血管内科病房,1.0,884160,2021-08-18 17:14:24,21-8-315_51-1_HTN.npy,0.0
111,4190.0,黄莉玲,37.0,女,内分泌糖尿病科病房,1.0,883488,2021-08-15 15:57:56,21-8-4190_37-0_HTN.npy,0.0
112,89.0,余繁斌,69.0,男,心血管内科病房,1.0,518243,2021-08-04 11:09:10,21-8-89_69-1_HTN.npy,0.0


In [47]:
HTN_errs

,Unnamed: 0,file,true,pred1,pred2,pred3,pred4,pred5,ERR_Count
7,7,21-1-1191_45-0_HTN.npy,1,0,0,0,0,0,5
14,14,21-1-1311_28-1_HTN.npy,1,0,0,0,0,0,5
24,24,21-1-1472_54-1_HTN.npy,1,0,0,0,0,0,5
25,25,21-1-1489_35-0_HTN.npy,1,0,0,0,0,0,5
43,43,21-1-1948_35-0_HTN.npy,1,1,1,0,0,0,3
...,...,...,...,...,...,...,...,...,...
1761,1761,21-8-272_51-1_HTN.npy,1,1,1,0,0,0,3
1803,1803,21-8-315_51-1_HTN.npy,1,0,0,0,0,0,5
1868,1868,21-8-4190_37-0_HTN.npy,1,0,0,0,0,0,5
1912,1912,21-8-89_69-1_HTN.npy,1,1,0,0,0,0,4


In [53]:
csv_data = pd.merge(csv_data,(HTN_errs.loc[:,['file','ERR_Count']].reset_index(drop=True).rename(columns={'file':'ecgFN'})),how='inner',on='ecgFN')
csv_data.to_csv('./ERR_fold/HTN/高血压分错.csv')

In [51]:
csv_data

,num,name,ages,gender,department,diagnose,ID,date,ecgFN,q_sum,ERR_Count
0,1191.0,李丽琴,45.0,女,内分泌糖尿病科病房,1.0,848037,2021-01-06 16:57:06,21-1-1191_45-0_HTN.npy,0.0,5
1,1311.0,李杰,28.0,男,心血管内科病房,1.0,849760,2021-01-19 09:40:16,21-1-1311_28-1_HTN.npy,0.0,5
2,1472.0,杨国富,54.0,男,心血管内科病房,1.0,851140,2021-01-26 19:29:04,21-1-1472_54-1_HTN.npy,0.0,5
3,1489.0,杨希,35.0,女,心血管内科病房,1.0,846397,2021-01-06 15:57:22,21-1-1489_35-0_HTN.npy,0.0,5
4,1948.0,林雪云,35.0,女,内分泌糖尿病科病房,1.0,415049,2021-01-05 15:46:46,21-1-1948_35-0_HTN.npy,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...
109,272.0,卢惟铭,51.0,男,内分泌糖尿病科病房,1.0,882533,2021-08-09 15:40:08,21-8-272_51-1_HTN.npy,0.0,3
110,315.0,叶奕斌,51.0,男,心血管内科病房,1.0,884160,2021-08-18 17:14:24,21-8-315_51-1_HTN.npy,0.0,5
111,4190.0,黄莉玲,37.0,女,内分泌糖尿病科病房,1.0,883488,2021-08-15 15:57:56,21-8-4190_37-0_HTN.npy,0.0,5
112,89.0,余繁斌,69.0,男,心血管内科病房,1.0,518243,2021-08-04 11:09:10,21-8-89_69-1_HTN.npy,0.0,4
